In [27]:
import lifetimes
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from lifetimes import BetaGeoFitter # BG/NBD
from lifetimes import GammaGammaFitter # Gamma-Gamma Model
from lifetimes.plotting import plot_frequency_recency_matrix

raw_data = pd.read_csv('RFMdata.csv')
df = raw_data.copy()
df.drop(df.columns[0],axis=1,inplace=True)
#df.drop_duplicates(inplace=True)
df.head()

C:\Users\27722\AppData\Local\Temp\ipykernel_7892\2402679309.py:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('RFMdata.csv')


,CustomerID,monetary_value,frequency,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,Date,rank,recency,total,Cluster
0,12346.0,189248.25,1,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,United Kingdom,2011-01-18 10:01:00,1,48,189248.25,3
1,12347.0,1240.30,31,537626,22212,FOUR HOOK WHITE LOVEBIRDS,6,2010-12-07 14:57:00,2.10,Iceland,2010-12-07 14:57:00,1,6,20.34,2
2,12347.0,1240.30,31,537626,84997C,BLUE 3 PIECE POLKADOT CUTLERY SET,6,2010-12-07 14:57:00,3.75,Iceland,2010-12-07 14:57:00,1,6,16.50,2
3,12347.0,1240.30,31,537626,85167B,BLACK GRAND BAROQUE PHOTO FRAME,30,2010-12-07 14:57:00,1.25,Iceland,2010-12-07 14:57:00,1,6,101.70,2
4,12347.0,1240.30,31,537626,21171,BATHROOM METAL SIGN,12,2010-12-07 14:57:00,1.45,Iceland,2010-12-07 14:57:00,1,6,40.68,2


In [28]:
#df = df[(df['Quantity']>0)]
df.drop(columns=['StockCode','Description','Country','Date'],inplace=True)
df.describe(include='all')

,CustomerID,monetary_value,frequency,InvoiceNo,Quantity,InvoiceDate,UnitPrice,rank,recency,total,Cluster
count,236270.000000,236270.000000,236270.000000,236270.0,236270.000000,236270,236270.000000,236270.0,236270.000000,230556.00000,236270.000000
unique,NaN,NaN,NaN,5782.0,NaN,5539,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,573585.0,NaN,2011-10-31 14:41:00,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,1114.0,NaN,1114,NaN,NaN,NaN,NaN,NaN
mean,16319.698265,2489.288268,188.650772,NaN,6.573797,NaN,4.872108,1.0,181.418716,26.02712,1.497778
std,2377.426691,3316.709034,198.715697,NaN,155.020351,NaN,45.837541,0.0,123.536067,455.81293,1.043254
min,12346.000000,0.000000,1.000000,NaN,1.000000,NaN,0.001000,1.0,0.000000,0.00100,0.000000
25%,14100.000000,665.960000,38.000000,NaN,1.000000,NaN,1.250000,1.0,63.000000,2.31000,1.000000
50%,16451.000000,1454.670000,114.000000,NaN,2.000000,NaN,2.460000,1.0,177.000000,5.79000,2.000000
75%,18532.000000,3188.250000,259.000000,NaN,6.000000,NaN,4.250000,1.0,302.000000,16.50000,2.000000


1. Calculate Average Order Value

In [29]:
df['avg_order_value']= df['total']/df['frequency']

df.sample(15)

,CustomerID,monetary_value,frequency,InvoiceNo,Quantity,InvoiceDate,UnitPrice,rank,recency,total,Cluster,avg_order_value
122007,16622.0,1512.22,333,550458,1,2011-04-18 13:13:00,5.79,1,138,1.65,1,0.004955
150172,17632.0,335.97,64,553766,1,2011-05-19 10:44:00,4.13,1,169,0.85,2,0.013281
103822,15785.0,2262.38,29,544309,10,2011-02-17 14:17:00,1.65,1,78,99.50,2,3.431034
8431,12581.0,423.35,15,573417,10,2011-10-31 08:56:00,0.79,1,334,4.20,0,0.280000
138489,17259.0,1093.35,87,537374,1,2010-12-06 12:55:00,5.95,1,5,1.25,2,0.014368
135446,17130.0,4340.55,163,551718,1,2011-05-03 16:06:00,0.83,1,153,2.95,1,0.018098
48287,13731.0,520.41,20,549188,4,2011-04-07 10:10:00,3.75,1,127,6.64,2,0.332000
223335,19715.0,4272.83,606,578347,1,2011-11-24 09:26:00,4.13,1,358,2.55,3,0.004208
146728,17503.0,264.33,23,573780,2,2011-11-01 11:12:00,8.25,1,335,3.30,0,0.143478
154583,17830.0,33.18,11,544835,12,2011-02-24 10:15:00,1.65,1,85,12.72,2,1.156364


In [30]:
total_customers = df.shape[0]

2. Calculate Purchase Frequency

In [31]:
purchase_frequency = sum(df['frequency'])/total_customers

3. Calculate Churn Rate

In [32]:
repeaters = df[df.frequency>1]
repeat_customers = repeaters.shape[0]
repeat_customers

235881

In [33]:
churn_rate = 1 - (repeat_customers/total_customers)

In [34]:
purchase_frequency,churn_rate

(188.65077242138233, 0.0016464214669658084)

### 4. Calculate Profit Margin

we assume a 5% profit margin for this, however this will be business specific

In [35]:
df['profit_margin'] = df['total']*0.05

## Customer Life Time Value

CLTV = (Avg Order Value * Purchase Frequency / Churn Rate) * Profit Margin

In [36]:
df['CLTV'] = (df['avg_order_value']*purchase_frequency/churn_rate)*df['profit_margin']

df.sample(15)

,CustomerID,monetary_value,frequency,InvoiceNo,Quantity,InvoiceDate,UnitPrice,rank,recency,total,Cluster,avg_order_value,profit_margin,CLTV
127986,16800.0,2438.40,164,579167,2,2011-11-28 14:06:00,2.08,1,362,8.26,1,0.050366,0.4130,2383.437778
151443,17691.0,117.80,9,537257,6,2010-12-06 11:10:00,2.95,1,5,29.70,2,3.300000,1.4850,561510.602722
147431,17537.0,913.88,61,543535,1,2011-02-09 13:37:00,4.95,1,70,1.25,2,0.020492,0.0625,146.749883
211413,19521.0,1842.80,183,574856,14,2011-11-07 12:06:00,4.13,1,341,69.30,1,0.378689,3.4650,150349.833516
119567,16517.0,176.22,17,545853,2,2011-03-07 13:50:00,2.55,1,96,4.20,2,0.247059,0.2100,5944.799770
56243,13975.0,1216.14,117,545978,12,2011-03-08 12:19:00,0.55,1,97,35.40,2,0.302564,1.7700,61363.233205
154062,17814.0,711.91,158,554512,1,2011-05-24 15:54:00,2.46,1,174,0.85,0,0.005380,0.0425,26.198012
20591,12972.0,4217.85,534,538177,1,2010-12-10 09:51:00,2.51,1,9,0.85,3,0.001592,0.0425,7.751472
19656,12946.0,967.17,111,538148,2,2010-12-09 16:26:00,4.21,1,8,7.50,2,0.067568,0.3750,2903.267951
171787,18457.0,2032.15,310,558774,1,2011-07-04 10:17:00,8.29,1,215,5.79,1,0.018677,0.2895,619.559479


In [37]:
df[['frequency','monetary_value']].corr()

,frequency,monetary_value
frequency,1.00000,0.61933
monetary_value,0.61933,1.00000


In [38]:
df.pivot_table(index='Cluster')

,CLTV,CustomerID,Quantity,UnitPrice,avg_order_value,frequency,monetary_value,profit_margin,rank,recency,total
Cluster,,,,,,,,,,,
0,3.929772e+06,16331.729433,7.047908,4.634335,1.514070,58.985082,809.848525,1.133490,1,283.447440,22.669801
1,9.878735e+07,16928.005987,6.562076,5.211369,2.509545,184.656863,2850.084360,1.716002,1,200.639158,34.320037
2,5.190813e+06,15304.328778,7.184699,4.480356,1.870221,53.254124,759.187970,1.183705,1,73.131825,23.674095
3,4.421582e+09,16952.945284,5.209631,5.224772,4.753987,525.490613,6253.907850,1.079655,1,206.931795,21.593096
